## Lab 3
Solve efficiently a generic n^2-1 puzzle (also known as Gen Puzzle, Boss Puzzle, Mystic Square) using path-search algorithms
- Quality: number of actions in the solution
- Cost: the total number of actions evaluated
- Efficiency: Quality vs. Cost

In [12]:
from collections import namedtuple
from random import choice
import numpy as np
from heapq import heappop, heappush
import matplotlib.pyplot as plt
import time
import tracemalloc


In [13]:
# direction: up, down, left, right
# pos1: position of the empty tail
# pos2: new possible position
action = namedtuple('Action', ['direction','pos1', 'pos2'])

## Utility functions

In [14]:
def display_state(state):
    """Visualize states"""
    for row in state:
        print(' '.join(f"{num:2}" if num != 0 else "  " for num in row))
    print()


def goal_state(PUZZLE_DIM):
    """ Creates the goal state for an n-puzzle"""
    goal = list(range(1, PUZZLE_DIM * PUZZLE_DIM)) + [0]
    return np.array([goal[i * PUZZLE_DIM:(i + 1) * PUZZLE_DIM] for i in range(PUZZLE_DIM)])


def precompute_goal_positions(goal_state):
    """Store the positions of each element according to the goal state"""
    positions = {}
    for i in range(goal_state.shape[0]):
        for j in range(goal_state.shape[1]):
            positions[goal_state[i, j]] = (i, j)
    return positions


def is_puzzle_solvable(state):
    """Determines if a given puzzle is solvable."""
    dim = state.shape[0]  

    def get_inv_count(puzzle):
        """Count inversions in the puzzle."""
        arr = [tile for row in puzzle for tile in row]
        inv_count = 0
        for i in range(dim * dim - 1):
            for j in range(i + 1, dim * dim):
                if arr[j] and arr[i] and arr[i] > arr[j]:
                    inv_count += 1
        return inv_count

    def find_x_position(puzzle):
        """Find the position of the blank tile (0) from the bottom."""
        for i in range(dim - 1, -1, -1): 
            for j in range(dim - 1, -1, -1): 
                if puzzle[i][j] == 0:
                    return dim - i 

    # Count inversions in the given puzzle
    inv_count = get_inv_count(state)

    # If grid is odd, 
    if dim % 2 != 0:
        # grid is odd
        return inv_count % 2 == 0 #return True if inversion count is even
    else:
        # grid is even
        pos = find_x_position(state)  # 0's position from the bottom
        if pos % 2 != 0:  # 0 is on an odd row from the bottom
            return inv_count % 2 == 0
        else:  # 0 is on an even row from the bottom
            return inv_count % 2 != 0
        

def initialization_puzzle(PUZZLE_DIM, RANDOMIZE_STEPS = 100_000):
    state = goal_state(PUZZLE_DIM)
    for _ in range(RANDOMIZE_STEPS):
        x, y = np.argwhere(state == 0)[0]
        state = do_action(state, choice(available_actions(state, (x, y))))
    return state 

## Handle Actions

In [15]:

def available_actions(state: np.ndarray, empty_pos) -> list['Action']:
    """Returns a list of available actions based on the empty tile's position."""
    # Puzzle dimension
    dim = state.shape[0]
    # Tile's position 
    x, y = int(empty_pos[0]), int(empty_pos[1])
    
    actions = list()
   
    if x > 0:
        # UP
        actions.append(action('up', (x, y), (x - 1, y)))
    if x < dim - 1:
        # DOWN
        actions.append(action('down', (x, y), (x + 1, y)))
    if y > 0:
        # LEFT
        actions.append(action('left',(x, y), (x, y - 1)))
    if y < dim - 1:
        # RIGHT
        actions.append(action('right', (x, y), (x, y + 1)))
    
    return actions


def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    """Applyes the selected action and returns the new state"""
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state


## A* algorithm

In [16]:
def a_star(start_state, goal_state, verbose=False):
    """Solves the n-puzzle problem using the A* algorithm"""

    state = start_state

    # Counter for tie-breaking in the priority queue
    counter = 0
    # Total cost (number of evaluated nodes/actions)
    evaluated_cost = 0

    # Initial empty tile position
    x, y = np.argwhere(state == 0)[0] 
    
    # Precompute goal positions for each element of the puzzle
    goal_positions = precompute_goal_positions(goal_state)
    
    # Priority queue for nodes to explore
    open_set = []
    # Set to store visited nodes
    visited = set()
    visited.add(state.tobytes())
    
    # Push the initial state into the priority queue with f = g + h (f = 0 + h)
    # (f, counter, current state, path, g)
    heappush(open_set, (0 + heuristic(state, goal_positions), counter, state, [(int(x), int(y))], 0))

    while open_set:
        # Get the state with the lowest f value
        f, _, current_state_bytes, path, g = heappop(open_set)

        #State in bytes back to a numpy array
        current_state = np.frombuffer(current_state_bytes, dtype=int).reshape(state.shape)


        if verbose:
            print(f"Path: {path}")
            display_state(current_state)
            print()

        # Check if goal state is reached
        if np.array_equal(current_state, goal_state):
            return path, evaluated_cost

        #New tail's position
        new_x, new_y = np.argwhere(current_state == 0)[0]
        # Available actions
        for action in available_actions(current_state, (new_x, new_y)):
            new_state = do_action(current_state, action)
            new_state_bytes = new_state.tobytes()
            
            if new_state_bytes not in visited:
                visited.add(new_state_bytes)

                # Compute heuristic
                h = heuristic(new_state, goal_positions)
                
                # New position and new path after the action
                new_pos = action.pos2
                new_path = path + [new_pos]
                # Push the new state into the priority queue
                heappush(open_set, (g + 1 + h, counter, new_state, new_path, g + 1))
                
                # Increment counter to break ties in case of equal f-values
                counter += 1
                evaluated_cost += 1

    # No solution found
    return [], float('inf') 
    

In [17]:
def measure_a_star(start_state, goal_state):
    """Run A* and measure efficiency metrics."""
    # Start memory tracking
    tracemalloc.start()
    # Start time tracking  
    start_time = time.perf_counter()  

    # Run A* algorithm
    path, evaluated_cost = a_star(start_state, goal_state)

    end_time = time.perf_counter()  
    # Get memory usage
    memory_usage, _ = tracemalloc.get_traced_memory()  
    tracemalloc.stop()

    return path, evaluated_cost, end_time - start_time, memory_usage / 1e6

### Heuristics

In [18]:
#*************** Possible Heuristics
def hamming_distance(current_state, goal_state):
    """Compute the Hamming distance between the current state and the goal state."""
    hamming = 0
    puzzle_dim = current_state.shape[0]
    for i in range(puzzle_dim):
        for j in range(puzzle_dim):
            if current_state[i][j] != goal_state[i][j] and current_state[i][j] != 0:
                hamming += 1
    return hamming


def manhattan_distance(state, goal_positions):
    """Computes the Manhattan distance between the current state and the goal state using pre-calculated goal positions of each element"""
    puzzle_dim = state.shape[0]
    distance = 0
    for r in range(puzzle_dim):
        for c in range(puzzle_dim):
            value = state[r, c]
            if value != 0:  # Skip the empty tile (element 0)
                goal_r, goal_c = goal_positions[value]
                distance += abs(goal_r - r) + abs(goal_c - c)
    return distance


def linear_conflict(state, goal_state):
    puzzle_dim = state.shape[0]
    conflict_count = 0

    # Row conflicts
    for row in range(puzzle_dim):
        for col1 in range(puzzle_dim):
            for col2 in range(col1 + 1, puzzle_dim):
                tile1 = state[row, col1]
                tile2 = state[row, col2]

                # Skip empty tile (0)
                if tile1 == 0 or tile2 == 0:
                    continue

                # Find the positions of the tiles in the goal state
                goal_pos1 = np.argwhere(goal_state == tile1)
                goal_pos2 = np.argwhere(goal_state == tile2)

                # Ensure both tiles are found in the goal state
                if goal_pos1.size == 0 or goal_pos2.size == 0:
                    continue

                goal_pos1 = goal_pos1[0]
                goal_pos2 = goal_pos2[0]

                # Check if they are in the same row and need to swap places
                if goal_pos1[0] == goal_pos2[0] == row and goal_pos1[1] > goal_pos2[1]:
                    conflict_count += 2  # Linear conflict adds 2 moves

    # Column conflicts
    for col in range(puzzle_dim):
        for row1 in range(puzzle_dim):
            for row2 in range(row1 + 1, puzzle_dim):
                tile1 = state[row1, col]
                tile2 = state[row2, col]

                # Skip empty tile (0)
                if tile1 == 0 or tile2 == 0:
                    continue

                # Find the positions of the tiles in the goal state
                goal_pos1 = np.argwhere(goal_state == tile1)
                goal_pos2 = np.argwhere(goal_state == tile2)

                # Ensure both tiles are found in the goal state
                if goal_pos1.size == 0 or goal_pos2.size == 0:
                    continue

                goal_pos1 = goal_pos1[0]
                goal_pos2 = goal_pos2[0]

                # Check if they are in the same column and need to swap places
                if goal_pos1[1] == goal_pos2[1] == col and goal_pos1[0] > goal_pos2[0]:
                    conflict_count += 2  # Linear conflict adds 2 moves

    return conflict_count


# Combine Manhattan distance with linear conflict
def combined_heuristic(state, goal):
    return manhattan_distance(state, goal) + 2 * linear_conflict(state, goal)



## manhattan_distance vs. combined_heuristics

I aim to compare the performance of using manhattan_distance versus combined_heuristics.

In [19]:
PUZZLE_DIM = 3
state = initialization_puzzle(PUZZLE_DIM, 100_000)
print("Initial state:")
display_state(state)
# Goal of the puzzle
goal = goal_state(PUZZLE_DIM)
print("Goal state:")
display_state(goal)

Initial state:
 5  4   
 8  1  6
 3  7  2

Goal state:
 1  2  3
 4  5  6
 7  8   



In [20]:
heuristic = manhattan_distance
_ , _ , solving_time, memory_usage = measure_a_star(state, goal)
print(f"Heuristic used: Manhattan distance")
print(f"Solving time: {solving_time:.2f} seconds")
print(f"Memory usage: {memory_usage:.2f}")
print()

heuristic = combined_heuristic
_ , _ , solving_time, memory_usage = measure_a_star(state, goal)
print(f"Heuristic used: combined heuristics")
print(f"Solving time: {solving_time:.2f} seconds")
print(f"Memory usage: {memory_usage}")

Heuristic used: Manhattan distance
Solving time: 0.22 seconds
Memory usage: 0.26

Heuristic used: combined heuristics
Solving time: 3.00 seconds
Memory usage: 0.110036


Based on the results, it appears that using the Manhattan Distance heuristic provides significantly faster execution time compared to the Combined Heuristics, though at the cost of higher memory usage.
For the upcoming tests, which will focus on solving puzzles of larger dimensions, I will prioritize the Manhattan Distance heuristic.

## TESTS

### Settings

In [21]:
PUZZLE_DIMs = [3, 4, 5, 6]
NUM_RAND_STEPS = [100_000, 1_000, 100, 100]
heuristic = manhattan_distance


In [ ]:
for PUZZLE_DIM, steps in zip(PUZZLE_DIMs, NUM_RAND_STEPS):
    print(f"Solving a {PUZZLE_DIM*PUZZLE_DIM-1}-puzzle")
    print("-------------------")

    # Initial state
    state = initialization_puzzle(PUZZLE_DIM, steps)
    print("Initial state:")
    display_state(state)

    # Goal of the puzzle
    goal = goal_state(PUZZLE_DIM)
    print("Goal state:")
    display_state(goal)

    # Check solvability
    if not is_puzzle_solvable(state):
        print("Puzzle is not solvable. Skipping...")
        continue

    
    start_time = time.time()
    path, cost = a_star(state, goal)
    solving_time = time.time() - start_time
   
    if path:
        print("Path:")
        print(path)
        print(f"Quality (number of actions in the solution): {len(path)}")
        print(f"Cost (total number of actions evaluated): {cost}")
    else:
        print("No solution found.")

    print(f"Solving time: {solving_time // 60} minutes and {solving_time % 60:.2f} seconds" if solving_time > 60 else f"Solving time: {solving_time:.2f} seconds")
    print()



Solving a 8-puzzle
-------------------
Initial state:
 8  2  7
 4     3
 1  5  6

Goal state:
 1  2  3
 4  5  6
 7  8   

Path:
[(1, 1), (1, 0), (0, 0), (0, 1), (0, 2), (1, 2), (1, 1), (1, 0), (2, 0), (2, 1), (1, 1), (0, 1), (0, 0), (1, 0), (1, 1), (1, 2), (2, 2), (2, 1), (2, 0), (1, 0), (1, 1), (2, 1), (2, 2)]
Quality (number of actions in the solution): 23
Cost (total number of actions evaluated): 1432
Solving time: 0.01 seconds

Solving a 15-puzzle
-------------------
Initial state:
14  8 10  9
13  3 12  7
 2  1  6 11
 5  4 15   

Goal state:
 1  2  3  4
 5  6  7  8
 9 10 11 12
13 14 15   

